In [ ]:
from datasets import load_dataset
from huggingface_hub import hf_hub_download
import pandas as pd

import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, IterableDataset, TensorDataset, Dataset, Subset
# import Subset

from tqdm.notebook import tqdm

import torch.nn.functional as F     
from latentmi import ksg
import numpy as np

In [2]:
file_path_metadata = hf_hub_download("recursionpharma/rxrx3-core", filename="metadata_rxrx3_core.csv",repo_type="dataset", local_dir="data")
rxrx3_core_metadata = pd.read_csv(file_path_metadata)
rxrx3_core = load_dataset("recursionpharma/rxrx3-core")

/tmp/ipykernel_1303745/2117931121.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  rxrx3_core_metadata = pd.read_csv(file_path_metadata)


Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/35 [00:00<?, ?it/s]

In [3]:
# # filter metadata for compounds only
# df = rxrx3_core_metadata[rxrx3_core_metadata['perturbation_type'] == 'COMPOUND'].copy()

# # create a label map for treatments
# treatments = sorted(df['treatment'].unique())
# lbl2idx = {t: i for i, t in enumerate(treatments)}

# # key: (experiment, plate, address) -> val: label_index
# lookup = {}
# for _, row in df.iterrows():
#     # normalize key components
#     key = (row['experiment_name'], str(row['plate']), row['address'])
#     lookup[key] = lbl2idx[row['treatment']]

# print(f"metadata ready! found {len(lookup)} valid wells :)")

#  filter metadata for compounds only
df = rxrx3_core_metadata[rxrx3_core_metadata['perturbation_type'] == 'COMPOUND'].copy()

# binary classification: control (0) vs perturbed (1)
df['is_control'] = (df['treatment'] == 'EMPTY_control').astype(int)

# key: (experiment, plate, address) -> val: binary_label (0=control, 1=perturbed)
lookup = {}
for _, row in df.iterrows():
    key = (row['experiment_name'], str(row['plate']), row['address'])
    lookup[key] = 0 if row['is_control'] else 1

print(f"metadata ready! found {len(lookup)} valid wells :)")
print(f"control samples: {sum(1 for v in lookup.values() if v == 0)}")
print(f"perturbed samples: {sum(1 for v in lookup.values() if v == 1)}")

metadata ready! found 95701 valid wells :)
control samples: 10234
perturbed samples: 85467


In [4]:
# class StreamingRxRx(IterableDataset):
#     def __init__(self, hf_ds, lookup_map, transform=None):
#         self.ds = hf_ds
#         self.lookup = lookup_map
#         self.tfm = transform

#     def __iter__(self):
#         buffer = []
#         current_site_key = None
        
#         for item in self.ds:
#             try:
#                 # key format: 'compound-001/Plate1/AA15_s1_1'
#                 # channel is the last digit (1-6)
#                 full_key = item['__key__']
#                 base_key, channel = full_key.rsplit('_', 1) 
                
#                 parts = base_key.split('/')
#                 exp = parts[0]
#                 plate = parts[1].replace('Plate', '')
#                 address = parts[2].split('_')[0]
                
#                 if (exp, plate, address) in self.lookup:
                    
#                     if current_site_key and base_key != current_site_key:
#                         buffer = []
                    
#                     current_site_key = base_key
#                     buffer.append(item['jp2'])
                    
#                     if len(buffer) == 6:
#                         label = self.lookup[(exp, plate, address)]
                        
#                         # stack images: [6, H, W]
#                         # convert each PIL image to tensor first
#                         tensors = [transforms.ToTensor()(img) for img in buffer]
#                         img_stack = torch.cat(tensors, dim=0) # cat along channel dim
                        
#                         if self.tfm:
#                             img_stack = self.tfm(img_stack)
                            
#                         yield img_stack, torch.tensor(label)
#                         buffer = [] # reset!
                        
#             except Exception:
#                 continue

# tfm = transforms.Compose([
#     transforms.Resize((224, 224)),
# ])

# stream_ds = StreamingRxRx(rxrx3_core['train'], lookup, transform=tfm)
# dl = DataLoader(stream_ds, batch_size=32)

In [5]:
# # quick check!
# images, labels = next(iter(dl))
# print(f"batch shape: {images.shape}")
# print(f"labels: {labels[:50]}") # check first 50 labels

In [6]:
# # configure split
# TRAIN_SPLIT = 0.5

# stream_ds = StreamingRxRx(rxrx3_core['train'], lookup, transform=tfm)

# imgs_all, lbls_all = [], []

# print('streaming full dataset...')
# for i, (x, y) in tqdm(enumerate(stream_ds)):
#     if np.random.rand() < 0.5: # subsample
#         continue
#     imgs_all.append(x)
#     lbls_all.append(y)

# # stack all tensors
# x_all = torch.stack(imgs_all)
# y_all = torch.stack(lbls_all)

# # calculate split index
# n_total = len(x_all)
# n_train = int(n_total * TRAIN_SPLIT)
# train_inds = np.random.choice(n_total, size=n_train, replace=False)

# print(f'total samples: {n_total}')
# print(f'train samples: {n_train} ({TRAIN_SPLIT*100}%)')
# print(f'test samples: {n_total - n_train} ({(1-TRAIN_SPLIT)*100}%)')

# # split the data
# x_train = x_all[train_inds]
# y_train = y_all[train_inds]
# test_inds = np.setdiff1d(np.arange(n_total), train_inds)
# x_test = x_all[test_inds]
# y_test = y_all[test_inds]

# # create dataloaders
# train_ds = TensorDataset(x_train, y_train)
# test_ds = TensorDataset(x_test, y_test)

# train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0)
# test_dl = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=0)

# print(f'done! train_dl has {len(train_dl)} batches, test_dl has {len(test_dl)} batches.')

In [7]:
class RxRxDataset(Dataset):
    def __init__(self, hf_ds, lookup_map, transform=None, subsample=0.1, n_imgs=10**3):
        self.hf_ds = hf_ds
        self.lookup = lookup_map
        self.tfm = transform
        
        print('indexing dataset...')
        self.samples = []
        buffer = []
        current_site_key = None
        
        for idx, item in tqdm(enumerate(hf_ds)):
            try:
                full_key = item['__key__']
                base_key, channel = full_key.rsplit('_', 1)
                
                parts = base_key.split('/')
                exp = parts[0]
                plate = parts[1].replace('Plate', '')
                address = parts[2].split('_')[0]
                
                if (exp, plate, address) in self.lookup:
                    if current_site_key and base_key != current_site_key:
                        buffer = []
                    
                    current_site_key = base_key
                    buffer.append(idx)
                    
                    if len(buffer) == 6:
                        label = self.lookup[(exp, plate, address)]
                        if np.random.rand() < subsample:
                            self.samples.append((buffer.copy(), label))
                        buffer = []

                    if len(self.samples) >= n_imgs:
                        break
            except Exception:
                continue
        
        print(f'found {len(self.samples)} valid sites!')
        
        # PRE-LOAD ALL IMAGES INTO MEMORY
        print('Loading all images into memory...')
        self.cached_data = []
        for channel_indices, label in tqdm(self.samples):
            tensors = []
            for ch_idx in channel_indices:
                img = hf_ds[ch_idx]['jp2']
                tensors.append(transforms.ToTensor()(img))
            
            img_stack = torch.cat(tensors, dim=0)
            if self.tfm:
                img_stack = self.tfm(img_stack)
            
            self.cached_data.append((img_stack, label))
        
        print(f'All {len(self.cached_data)} images cached in memory!')
    
    def __len__(self):
        return len(self.cached_data)
    
    def __getitem__(self, idx):
        return self.cached_data[idx]

tfm = transforms.Compose([
    transforms.Resize((224, 224)),])


full_ds = RxRxDataset(rxrx3_core['train'], lookup, transform=tfm, subsample=1, n_imgs=10**4)

# Split into train/test
n_total = len(full_ds)
n_train = int(n_total)# * 0.5)
train_inds = np.random.choice(n_total, size=n_train, replace=False)
test_inds = np.setdiff1d(np.arange(n_total), train_inds)

print(f'total samples: {n_total}')
print(f'train samples: {n_train}')
print(f'test samples: {len(test_inds)}')

train_ds = Subset(full_ds, train_inds)
test_ds = Subset(full_ds, test_inds)

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=0)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=0)

indexing dataset...


0it [00:00, ?it/s]

found 10000 valid sites!
Loading all images into memory...


  0%|          | 0/10000 [00:00<?, ?it/s]

All 10000 images cached in memory!
total samples: 10000
train samples: 10000
test samples: 0


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# noise helpers
def add_gauss(x, sigma=0.1):
    return x + torch.randn_like(x) * sigma

def pixelate(x, scale=4):
    # average tiles to drop res, then upsample back
    h, w = x.shape[-2:]
    small = F.avg_pool2d(x, kernel_size=scale, stride=scale)
    return F.interpolate(small, size=(h, w), mode='nearest')

def reinit_model():
    m = models.mobilenet_v3_small(weights='DEFAULT')
    m.features[0][0] = nn.Conv2d(6, 16, kernel_size=3, stride=2, padding=1, bias=False)
    m.classifier[3] = nn.Linear(m.classifier[3].in_features, 2)  # binary classification
    return m.to(device)

# def run_experiment(noise_fn, tag, n_epochs=50):
#     print(f"starting run: {tag} :)")
#     net = reinit_model()
#     opt = torch.optim.Adam(net.parameters(), lr=1e-3)
#     crit = nn.CrossEntropyLoss()
    
#     net.train()
#     for epoch in range(n_epochs):
#         epoch_loss = 0.0
#         for i, (x, y) in enumerate(train_dl):
#             x, y = x.to(device), y.to(device)
#             if noise_fn: x = noise_fn(x) # inject noise!
            
#             opt.zero_grad()
#             loss = crit(net(x), y)
#             loss.backward()
#             opt.step()
#             epoch_loss += loss.item()
#         print(f"Epoch {epoch+1}/{n_epochs} | Loss: {epoch_loss/len(train_dl):.4f}")
            
#     # save
#     fname = f"images/models/model_{tag}.pt"
#     torch.save(net.state_dict(), fname)
#     print(f"saved weights to {fname}!")
    
#     # eval & mi calc (binary classification)
#     net.eval()
#     preds, targets = [], []
    
#     with torch.no_grad():
#         for i, (x, y) in enumerate(train_dl):

#             x = x.to(device)
#             if noise_fn: x = noise_fn(x)
            
#             p = net(x).argmax(dim=1).cpu()
#             preds.append(p)
#             targets.append(y)

#     y_hat = torch.cat(preds).numpy()
#     y_true = torch.cat(targets).numpy()

#     print(y_hat[:10], y_true[:10])
    
#     # calculate MI for binary classification
#     mi_score = ksg.midd(y_hat.reshape(-1, 1), y_true.reshape(-1, 1))
    
#     print(f"MI score: {mi_score:.4f}")
#     return mi_score


# experiments = [(None, "clean")] + [(lambda x, s=s: add_gauss(x, s), f"gauss_{s}") for s in np.logspace(-3, 0, 10)] + [(lambda x, sc=sc: pixelate(x, sc), f"pix_{sc}x") for sc in 2**np.arange(1, 7)]

# results = {}
# for func, name in experiments:
#     results[name] = run_experiment(func, name)

In [9]:
from latentmi import lmi

def run_experiment(noise_fn, tag, n_epochs=50):
    print(f"starting run: {tag} :)")
    net = reinit_model()
    opt = torch.optim.Adam(net.parameters(), lr=1e-3)
    crit = nn.CrossEntropyLoss()
    
    net.train()
    for epoch in range(n_epochs):
        epoch_loss = 0.0
        for i, (x, y) in enumerate(train_dl):
            x, y = x.to(device), y.to(device)
            if noise_fn: x = noise_fn(x) # inject noise!
            
            opt.zero_grad()
            loss = crit(net(x), y)
            loss.backward()
            opt.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{n_epochs} | Loss: {epoch_loss/len(train_dl):.4f}")
            
    # save
    fname = f"images/models/model_{tag}.pt"
    torch.save(net.state_dict(), fname)
    print(f"saved weights to {fname}!")
    
    # eval & extract last layer representations
    net.eval()
    representations, targets = [], []
    
    # Create a hook to extract features from the last layer before classifier
    features = {}
    def get_features(name):
        def hook(model, input, output):
            features[name] = output.detach()
        return hook
    
    # Register hook on the last layer before classification head
    # For MobileNetV3, this is the last feature layer
    net.classifier[0].register_forward_hook(get_features('last_layer'))
    
    with torch.no_grad():
        for i, (x, y) in enumerate(train_dl):
            x = x.to(device)
            if noise_fn: x = noise_fn(x)
            
            _ = net(x)  # forward pass to trigger hook
            representations.append(features['last_layer'].cpu())
            targets.append(y)

    # Convert to numpy arrays
    X = torch.cat(representations).numpy()  # shape: (n_samples, feature_dim)
    Y = torch.cat(targets).numpy().reshape(-1, 1)  # shape: (n_samples, 1)
    
    print(f"Representations shape: {X.shape}, Labels shape: {Y.shape}")
    print(f"Sample labels: {Y[:10].ravel()}")
    
    # Calculate MI using lmi
    pmis, _, _ = lmi.estimate(X, Y, validation_split=0.3, batch_size=512, epochs=50, quiet=False)
    mi_score = np.nanmean(pmis)
    
    print(f"MI score: {mi_score:.4f}")
    return mi_score


experiments = [(None, "clean")] + [(lambda x, s=s: add_gauss(x, s), f"gauss_{s}") for s in np.logspace(-3, 0, 10)] + [(lambda x, sc=sc: pixelate(x, sc), f"pix_{sc}x") for sc in 2**np.arange(1, 7)]

results = {}
for func, name in experiments:
    results[name] = run_experiment(func, name)

starting run: clean :)
Epoch 1/50 | Loss: 0.2700
Epoch 2/50 | Loss: 0.2362
Epoch 3/50 | Loss: 0.2137
Epoch 4/50 | Loss: 0.1744
Epoch 5/50 | Loss: 0.1363
Epoch 6/50 | Loss: 0.0930
Epoch 7/50 | Loss: 0.0678
Epoch 8/50 | Loss: 0.0456
Epoch 9/50 | Loss: 0.0393
Epoch 10/50 | Loss: 0.0254
Epoch 11/50 | Loss: 0.0285
Epoch 12/50 | Loss: 0.0283
Epoch 13/50 | Loss: 0.0177
Epoch 14/50 | Loss: 0.0200
Epoch 15/50 | Loss: 0.0234
Epoch 16/50 | Loss: 0.0166
Epoch 17/50 | Loss: 0.0233
Epoch 18/50 | Loss: 0.0147
Epoch 19/50 | Loss: 0.0154
Epoch 20/50 | Loss: 0.0170
Epoch 21/50 | Loss: 0.0105
Epoch 22/50 | Loss: 0.0120
Epoch 23/50 | Loss: 0.0155
Epoch 24/50 | Loss: 0.0191
Epoch 25/50 | Loss: 0.0088
Epoch 26/50 | Loss: 0.0090
Epoch 27/50 | Loss: 0.0141
Epoch 28/50 | Loss: 0.0175
Epoch 29/50 | Loss: 0.0075
Epoch 30/50 | Loss: 0.0091
Epoch 31/50 | Loss: 0.0145
Epoch 32/50 | Loss: 0.0111
Epoch 33/50 | Loss: 0.0141
Epoch 34/50 | Loss: 0.0130
Epoch 35/50 | Loss: 0.0084
Epoch 36/50 | Loss: 0.0252
Epoch 37/50 | 

In [10]:
results

{'clean': 0.4811381890983721,
 'gauss_0.001': 0.4695656449831164,
 'gauss_0.0021544346900318847': 0.4903326613483761,
 'gauss_0.004641588833612777': 0.42160580930826164,
 'gauss_0.01': 0.4776842258294589,
 'gauss_0.021544346900318832': 0.49039679639687833,
 'gauss_0.046415888336127774': 0.46511012330151064,
 'gauss_0.09999999999999999': 0.47368794735794606,
 'gauss_0.21544346900318823': 0.44400605097548485,
 'gauss_0.4641588833612777': 0.2084052759289691,
 'gauss_1.0': 0.060654907877271985,
 'pix_2x': 0.4689470651190459,
 'pix_4x': 0.47657552015923926,
 'pix_8x': 0.4736896925068049,
 'pix_16x': 0.46293073183836775,
 'pix_32x': 0.28368668331958,
 'pix_64x': 0.12649970056508966}